In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.1 MB/s eta 0:00:00


In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.537s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [9]:
df_path = '/content/drive/MyDrive/wip3.csv'

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

df = pd.read_csv(df_path)
df['type'].value_counts()
df['target_model'] = (df['target']>0).astype(int)

lr = LogisticRegression()
lr.fit(df[['bert_pred', 'roberta_pred', 'mental_bert_pred', 'mental_roberta_pred']], df['target_model'])


LogisticRegression()

In [4]:
df['all_pred'] = lr.predict(df[['bert_pred', 'roberta_pred', 'mental_bert_pred', 'mental_roberta_pred']])

In [5]:
df.to_csv('test_wip3.csv')

In [6]:
%%writefile CONFIG.py

model_dict = {'Bert':'bert_pred', 'RoBertA':'roberta_pred', 'MENTAL-Bert':'mental_bert_pred', 'MENTAL-RoBertA':'mental_roberta_pred', 'Ensemble':'all_pred'}
path = '/content/test_wip3.csv'
target_col = 'target_model'
mod_col = 'type'
no_flag = 'NOT_FLAGGED'
hand_flag = 'MODERATOR'
sys_flag = 'STRING_MATCH'
text = 'text'


Writing CONFIG.py


In [7]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import CONFIG


if 'new_targets' not in st.session_state.keys():
  st.session_state['new_targets'] = dict()

df = pd.read_csv(CONFIG.path)
#model_list = ['Bert', 'RoBertA', 'MENTAL-Bert', 'MENTAL-RoBertA', 'Ensemble']
st.title('Noise Removing Hand Tool')

st.markdown("""
 * Use the menu at left to select data and request new random text
 * The random text will appear below, along with its grounf truth target, and model predictions
 * Write the target you would predict for the text as a moderator
""")

st.sidebar.markdown("## Select the model(s) and data for noise detection")

select_model_event = st.sidebar.selectbox('Which Model to Use?',
                                    CONFIG.model_dict.keys())

model_outputs = df[CONFIG.model_dict[select_model_event]]
targets = df[CONFIG.target_col]

select_data_event = st.sidebar.selectbox('Which Data to Sample?',
                                    ['All', 'All Moderated', 'Hand Moderated', 'System Moderated', 'Not Moderated'])

mask1 = pd.Series(np.ones(len(df)).astype(bool))

if select_data_event == 'All Moderated':
  mask1 = (df[CONFIG.mod_col] != CONFIG.no_flag)

elif select_data_event == 'Hand Moderated':
  mask1 = (df[CONFIG.mod_col] == CONFIG.hand_flag)

elif select_data_event == 'System Moderated':
  mask1 = (df[CONFIG.mod_col] == CONFIG.sys_flag)

elif select_data_event == 'Not Moderated':
  mask1 = (df[CONFIG.mod_col] == CONFIG.no_flag)

select_outcome_event = st.sidebar.selectbox('Which Prediction Outcome to Sample?',
                                    ['All', 'True Positive', 'True Negative', 'False Positive', 'False Negative'])

mask2 = pd.Series(np.ones(len(df)).astype(bool))

if select_outcome_event == 'True Positive':
  mask2 = ((targets == 1) & (model_outputs == 1))
if select_outcome_event == 'True Negative':
  mask2 = ((targets == 0) & (model_outputs == 0))
if select_outcome_event == 'False Positive':
  mask2 = ((targets == 0) & (model_outputs == 1))
if select_outcome_event == 'False Negative':
  mask2 = ((targets == 1) & (model_outputs == 0))

mask = (mask1 & mask2)

sample_df = df[mask]

if st.sidebar.button('Pick New Random Page'):
  rand_ind = sample_df.sample().index[0]
  st.session_state['rand_ind'] = rand_ind

if 'rand_ind' in st.session_state.keys():
  rand_ind = st.session_state['rand_ind']
else:
  rand_ind = 0

st.markdown(sample_df.loc[rand_ind, CONFIG.text])
st.markdown(f'Ground Truth Target: {sample_df.loc[rand_ind, CONFIG.target_col]}')

for model_name in CONFIG.model_dict:
  st.markdown(f'{model_name} prediction: {sample_df.loc[rand_ind, CONFIG.model_dict[model_name]]}')

new_target = st.selectbox('What is the appropriate target?',
                  ['<select>',0, 1])

if st.button('Save New Target'):
  st.session_state['new_targets'][rand_ind] = new_target

st.sidebar.download_button('Save All New Targets', pd.Series(st.session_state['new_targets']).to_csv().encode('utf-8'),
                              file_name='new_targets.csv', mime='text/csv')

Writing app.py


In [8]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & wget -q -O - ipv4.icanhazip.com

104.196.52.71
npx: installed 22 in 4.901s
your url is: https://few-parrots-arrive.loca.lt
